In [1]:
%reload_ext Cython

In [2]:
import cProfile
import tensorflow as tf
import os
import re
import chess
import test
import boardclass
import importlib
import numpy as np
import pickle
import training
importlib.reload(boardclass)
importlib.reload(test)
importlib.reload(training)
#takes a long time to load, so only run once
data_size = 100

In [3]:
#train_data = training.TrainingData_PQR("../extracted/good_games.txt",data_size)
#bin_file = open("pickled_train_data.bin",mode='wb')
#pickle.dump(train_data,bin_file)
#bin_file.close()

In [4]:
bin_file = open("pickled_train_data.bin",mode='rb')
train_data = pickle.load(bin_file)
bin_file.close()

In [5]:
tf.reset_default_graph()
importlib.reload(training)
board = boardclass.ChessBoard(-1)
board.train_model(data_size,train_data,1)
board.restore_model()

score P: [[ 1186.89697266]]
score Q: [[-2070.71386719]]
score R: [[ 1246.40979004]]
Time used 2.5090649127960205
Model saved at model_final
INFO:tensorflow:Restoring parameters from ./model_final


In [ ]:
test.start_game(-1,clear=True,against_AI=True,b=board)

8	_	Rook	Bishop	Queen	King	Bishop	_	Rook	

7	_	_	Pawn	_	Pawn	_	_	Pawn	

6	_	_	_	_	_	Knight	Pawn	_	

5	_	_	_	Pawn	_	Pawn	_	_	

4	_	_	_	_	_	_	_	_	

3	_	_	_	_	_	_	_	_	

2	_	Pawn	Pawn	Pawn	Pawn	Pawn	Pawn	Pawn	

1	Rook	Knight	Bishop	Queen	King	Bishop	Knight	Rook	

	a	b	c	d	e	f	g	h	
['1', 'Queen']
